# Importo i dati dal file e concateno la stringa per poterla poi riscrivere

In [30]:
fname = 'acquisizione_dati_grezzi_400ms.txt'

with open(fname, 'r') as file:
    misura = []
    tempo_di_acquisizione = []
    numero_di_bytes_in_ingresso = []
    stringa_bytes = []
    
     # Flag per saltare la prima riga
    prima_riga = True

    # Legge il file riga per riga
    for line in file:
        # Salta la prima riga
        if prima_riga:
            prima_riga = False
            continue
        
        columns = line.strip().split('\t')

        misura.append(columns[0])
        tempo_di_acquisizione.append(columns[1])
        numero_di_bytes_in_ingresso.append(columns[2])
        stringa_bytes.append(columns[3])

stringa_bytes_concatenata = ''.join(stringa_bytes)

# Ricerco la posizione della parola di controllo all'interno della mia stringa concatena

In [31]:
posizioni_aaaa = []
posizione = stringa_bytes_concatenata.find("aaaa")

while posizione != -1:
    posizioni_aaaa.append(posizione)
    posizione = stringa_bytes_concatenata.find("aaaa", posizione + 1)

# Estrazione temperatura ed umidità e conversione in decimale

In [32]:
temperatura_convertita = []
umidita_relativa_convertita = []

for i in posizioni_aaaa:
    umidita = str(int(''.join(stringa_bytes_concatenata[i-8:i-4]), 16))
    temperatura = str(int(''.join(stringa_bytes_concatenata[i-4:i]),16))
    umidita_relativa_convertita.append(umidita)
    temperatura_convertita.append(temperatura)

# Creazione di un file con i dati convertiti in decimale

In [33]:
fname = "Dati_400ms_ordinati"
with open(fname, "w", encoding="utf-8") as file:
    dati_in_colonna = ['misura', 'tempo di acquisizione', 'umidità relativa', 'temperatura']
    file.write("\t".join(dati_in_colonna) + "\n")
    

with open(fname, "a", encoding="utf-8") as file:
    for i in range(len(misura)):
        riga = [str(misura[i]), str(tempo_di_acquisizione[i]), str(umidita_relativa_convertita[i]), str(temperatura_convertita[i])]
        file.write("\t".join(riga) + "\n")

# Conversione dei dati in valori utili e non pernacchie

In [34]:
umidita_relativa_lineare = [] 
umidita_relativa_true = []
temperatura = []

for i in umidita_relativa_convertita[1:]:
    a, b, c = -2.0468, 0.0367, -1.5955E-6
    relative_humidity = round((a + b*int(i)+ c*int(i)**2), 3)
    umidita_relativa_lineare.append(relative_humidity)
    
    
for i in temperatura_convertita[1:]:
    a, b = -39.7, 0.01
    temperature = round((a + b*int(i)), 3)
    temperatura.append(temperature)

    
for i in range(len(umidita_relativa_lineare)):
    t_1, t_2 = 0.01, 0.00008
    relative_humidity_true = round((temperatura[i]-25)*(t_1+t_2*int(umidita_relativa_convertita[i+1]))+umidita_relativa_lineare[i],3)
    umidita_relativa_true.append(relative_humidity_true)

# Creo il file contenente i dati convertiti 

In [35]:
fname = "Dati_400ms_convertiti"
with open(fname, "w", encoding="utf-8") as file:
    dati_in_colonna = ['Misura', 'Tempo di acquisizione (sec)', 'Umidità relativa (%)', 'Temperatura (°C)']
    file.write("\t".join(dati_in_colonna) + "\n")
    

with open(fname, "a", encoding="utf-8") as file:
    for i in range(len(misura)):
        riga = [str(misura[i]), str(tempo_di_acquisizione[i]), str(umidita_relativa_true[i]), str(temperatura[i])]
        file.write("\t".join(riga) + "\n")